In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
from langchain_core.prompts import ChatPromptTemplate

template = """Based on the {context}, identify the entities and relations, and write the corresponding Cypher query to create the knowledge graph in details.Use the following format for the query:

`CREATE (node1:Label1 {{property1: "value1", property2: "value2", ...}})-[relationship:RELATIONSHIP_TYPE {{propertyA: "valueA", propertyB: "valueB", ...}}]->(node2:Label2 {{property3: "value3", property4: "value4", ...}})`

Convert the context into nodes and relationships using the Cypher language. The output should only contain the complete queries.

Neo4j:
"""
prompt = ChatPromptTemplate.from_template(template)


In [24]:
from langchain_openai import OpenAI
from langchain_core.output_parsers import StrOutputParser

model = OpenAI()
output_parser = StrOutputParser()

In [25]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("files/中華郵政股份有限公司_資訊整體架構健檢及優化服務專案_建議書徵求說明書0420.pdf")
pages = loader.load_and_split()

In [27]:
content = ''
for i in pages[:5]:
    content += i.page_content
content

'資訊整體架構健檢及優化服務\n專案 \n建議書徵求說明書  \n(Request for Proposal )  \n \n \n \n \n \n \n \n108年11月25日\n交通部中華郵政股份有限公司「資訊整體架構健檢及優化服務 專案」建議書徵求說明書   \n \nI \n交通部中華郵政股份有限公司  \n目錄  \n壹、 概述  ................................ ................................ ................................ ....................  1 \n一、  專案緣起  ................................ ................................ ................................ ...............................  1 \n二、  專案目標  ................................ ................................ ................................ ...............................  1 \n貳、 文件詞彙與用語  ................................ ................................ ................................  2 \n一、  名詞定義  ................................ ................................ ................................ ...............................  2 \n二、  文件用語  ................................ ................................ ................................ ...............................  4 \n參

In [30]:
chain = prompt | model | output_parser
neo4j_syntax = chain.invoke({"context": content})
neo4j_syntax

'\nCREATE (Company:Label1 {name: "交通部中華郵政股份有限公司"})-[CONDUCTS:RELATIONSHIP_TYPE {project_name: "資訊整體架構健檢及優化服務 專案", start_date: "108年11月25日"}]->(Proposal:Label2 {name: "建議書徵求說明書", date: "108年11月25日"});\nCREATE (Proposal)-[:CONTAINS]->(Overview:Label3 {name: "概述"});\nCREATE (Overview)-[:CONTAINS]->(Project_Rationale:Label4 {name: "專案緣起", start_date: "108年11月25日"});\nCREATE (Overview)-[:CONTAINS]->(Project_Goal:Label4 {name: "專案目標"});\nCREATE (Proposal)-[:CONTAINS]->(Terminology:Label3 {name: "文件詞彙與用語"});\nCREATE (Terminology)-[:CONTAINS]->(Definitions:Label4 {name: "名詞定'

In [31]:
queries = [line.strip() for line in neo4j_syntax.split('\n') if line.strip()]
queries

['CREATE (Company:Label1 {name: "交通部中華郵政股份有限公司"})-[CONDUCTS:RELATIONSHIP_TYPE {project_name: "資訊整體架構健檢及優化服務 專案", start_date: "108年11月25日"}]->(Proposal:Label2 {name: "建議書徵求說明書", date: "108年11月25日"});',
 'CREATE (Proposal)-[:CONTAINS]->(Overview:Label3 {name: "概述"});',
 'CREATE (Overview)-[:CONTAINS]->(Project_Rationale:Label4 {name: "專案緣起", start_date: "108年11月25日"});',
 'CREATE (Overview)-[:CONTAINS]->(Project_Goal:Label4 {name: "專案目標"});',
 'CREATE (Proposal)-[:CONTAINS]->(Terminology:Label3 {name: "文件詞彙與用語"});',
 'CREATE (Terminology)-[:CONTAINS]->(Definitions:Label4 {name: "名詞定']

In [32]:
import os 
from neo4j import GraphDatabase

# Connect to the Neo4j database
URL = os.getenv("URL")
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")

driver = GraphDatabase.driver(URL, auth=(USERNAME, PASSWORD))

# Function to execute a write query
def write_query(tx, query):
    tx.run(query)

# Execute the queries
with driver.session() as session:
    for query in queries:
        try:
            session.execute_write(write_query, query)
        except:
            print(f"Error with query: {query}")
# Close the connection
driver.close()

Error with query: CREATE (Terminology)-[:CONTAINS]->(Definitions:Label4 {name: "名詞定


In [35]:
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI

graph = Neo4jGraph(url=URL, username=USERNAME, password=PASSWORD)

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

chain.run("交通部中華郵政股份有限公司有什麼概述")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:Label1)-[:RELATIONSHIP_TYPE]->(m:Label2)
WHERE m.name = '交通部中華郵政股份有限公司'
RETURN n
Full Context:
[]

> Finished chain.


'我不知道答案。'